### Setup

In [ ]:
import os

import azure.identity
import dotenv
import openai

import warnings
warnings.filterwarnings('ignore')

from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

dotenv.load_dotenv()

azure_credential = azure.identity.DefaultAzureCredential()

# Initialize Azure OpenAI client
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_API_ENDPOINT")
AZURE_OPENAI_ADA_DEPLOYMENT = os.getenv("AZURE_OPENAI_ADA_DEPLOYMENT")

token_provider = azure.identity.get_bearer_token_provider(azure_credential, "https://cognitiveservices.azure.com/.default")
openai_client = openai.AzureOpenAI(
    api_version="2023-07-01-preview",
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com",
    azure_ad_token_provider=token_provider)

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_ADA_DEPLOYMENT, input=text)
    return get_embeddings_response.data[0].embedding

# Initialize Azure search client
AZURE_SEARCH_SERVICE = os.getenv("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_ENDPOINT = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"

AZURE_SEARCH_FULL_INDEX = "gptkbindex"
search_client = SearchClient(AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_FULL_INDEX, credential=azure_credential)


#### Vector search is not always optimal

For example, consider searches for exact strings.

In [ ]:
search_query = "$45.00"
search_vector = get_embedding(search_query)
r = search_client.search(
    None, 
    top=3, 
    vector_queries=[
        VectorizedQuery(vector=search_vector,k_nearest_neighbors=50, fields="embedding")]
    )
for doc in r:
    found_content = "Not found"
    if search_query in doc['content']:
        found_content = doc['content'][doc['content'].find(search_query):].replace("\n", " ")
    print(f"Score: {doc['@search.score']:.5f}\tMatching text: {found_content}")

Compare to a text search for same query:

In [ ]:
r = search_client.search(search_query, top=3)
for doc in r:
    found_content = "Not found"
    if search_query in doc['content']:
        found_content = doc['content'][doc['content'].find(search_query):].replace("\n", " ")
    print(f"Score: {doc['@search.score']:.5f}\tMatching text: {found_content}")

#### Hybrid retrieval

Uses RRF to combine vector and text results.

In [ ]:
search_vector = get_embedding(search_query)
r = search_client.search(
    search_query, 
    top=15, 
    vector_queries=[
        VectorizedQuery(vector=search_vector, k_nearest_neighbors=10, fields="embedding")]
    )
for doc in r:
    found_content = "Not found"
    if search_query in doc['content']:
        found_content = doc['content'][doc['content'].find(search_query):].replace("\n", " ")
    print(f"Score: {doc['@search.score']:.5f}\tMatching text: {found_content}")

#### Hybrid ranking is not always optimal

In [ ]:
search_query = "learning about underwater activities"
search_vector = get_embedding(search_query)
r = search_client.search(
    search_query, 
    top=5, 
    vector_queries=[
        VectorizedQuery(vector=search_vector, k_nearest_neighbors=10, fields="embedding")]
    )
for doc in r:
    content = doc["content"].replace("\n", " ")[:150]
    print(f"Score: {doc['@search.score']:.5f}\t\tContent: {content}")

#### Hybrid + semantic reranking 🎉

In [ ]:
search_query = "learning about underwater activities"
search_vector = get_embedding(search_query)
r = search_client.search(
        search_query,
        top=5, 
        vector_queries=[
                VectorizedQuery(vector=search_vector, k_nearest_neighbors=50, fields="embedding")],
        query_type="semantic",
        semantic_configuration_name="default")

for doc in r:
    content = doc["content"].replace("\n", " ")[:150]
    print(f"Score: {doc['@search.score']:.5f}\tReranker: {doc['@search.reranker_score']:.5f}\tContent: {content}")